In [1]:
from kaggle_environments import make, evaluate, utils
import numpy as np
import random
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from gym import spaces

Loading environment lux_ai_2022 failed: No module named 'vec_noise'


<img src="./boardexample.png" alt="Drawing" style="width: 200px;"/>
obs.board property in this board will be

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 1, 2, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 2, 1, 2, 0, 2, 0]

I don't know where to start than i will copy another code with comments

In [9]:

class ConnectFourGym:

    def __init__(self, agent2="random"):
        ks_env = make('connectx', debug=True)
        self.env = ks_env.train([None, agent2])
        self.rows = ks_env.configuration.rows
        self.columns = ks_env.configuration.columns
        #We define actions possible here
        self.action_space = spaces.Discrete(self.columns)
        #We define all of space we are playing
        self.observation_space = spaces.Box(low=0,
                                            high=2,
                                            shape=(self.rows, self.columns, 1),
                                            dtype=float)
        # Tuple corresponding to the min and max possible rewards
        self.reward_range = (-10, 1)
        # StableBaselines throws error if these are not defined
        self.spec = None
        self.metadata = None

    def reset(self):
        self.obs = self.env.reset()
        return self.obs['board']

    def change_reward(self, old_reward, done):
        if old_reward == 1:
            return 1
        elif done:
            return -1
        else:
            return 0

    def step(self, action):
        is_valid = (self.obs['board'][int(action)] == 0)
        if is_valid:
            self.obs, old_reward, done, _ = self.env.step(int(action))
            reward = self.change_reward(old_reward, done)
        else:
            reward, done, _ = -10, True, {}
        return self.obs['board'],reward,done

I am getting state action reward values with any agent

Next i will use q learning 
- I will use algorithm from this youtube video
- https://www.youtube.com/watch?v=wc-FxNENg9U

<img src="./loss.png" alt="Drawing"/>

Lets define constants for changing if necessary

In [13]:
class DeepQNetwork(nn.Module):
    def __init__(self, lr,input_dims, fc1_dims, fc2_dims,n_actions):
        super(DeepQNetwork,self).__init__()
        self.input_dims = input_dims
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.n_actions = n_actions
        self.fc1 = nn.Linear(*self.input_dims,self.fc1_dims)
        self.fc2 = nn.Linear(self.fc1_dims, self.fc2_dims)
        self.fc3 = nn.Linear(self.fc2_dims, self.n_actions)
        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.loss = nn.MSELoss()
        self.device = T.device('cpu')
        self.to(self.device)
    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        actions = self.fc3(x)
        
        return actions

class Agent():
    def __init__(self, gamma, epsilon, lr, input_dims, batch_size,
                     n_actions, max_mem_size=100000, eps_end=0.01, eps_dec=5e-4):
        self.gamma = gamma
        self.epsilon = epsilon
        self.eps_min = eps_end
        self.eps_dec = eps_dec
        self.lr = lr
        self.action_space = [i for i in range(n_actions)]
        self.mem_size = max_mem_size
        self.batch_size = batch_size
        self.mem_cntr = 0
            
        self.Q_eval = DeepQNetwork(self.lr, n_actions = n_actions,
                                       input_dims=input_dims, fc1_dims = 256, fc2_dims= 256)
        self.state_memory = np.zeros((self.mem_size, *input_dims), dtype=np.float32)
        self.new_state_memory = np.zeros((self.mem_size, *input_dims), dtype=np.float32)
        self.action_memory = np.zeros(self.mem_size, dtype=np.int32)
        self.reward_memory = np.zeros(self.mem_size, dtype=np.float32)
        self.terminal_memory = np.zeros(self.mem_size, dtype=bool)
    def store_transition(self, state, action, reward, state_, done):
        index = self.mem_cntr % self.mem_size
        self.state_memory[index] = state
        self.new_state_memory[index] = state_
        self.reward_memory[index] = reward
        self.action_memory[index] = action
        self.terminal_memory[index] = done
            
        self.mem_cntr += 1

    def choose_action(self,observation):
        if np.random.random() > self.epsilon:
            state = T.tensor([observation]).to(self.Q_eval.device)
            actions = self.Q_eval.forward(state)
            action = T.argmax(actions).item() ##converts tensor action to int
        else:
            action = np.random.choice(self.action_space)
            
        return action
        
    def learn(self):
        if self.mem_cntr < self.batch_size: ##Printlemiyorsa bunun yuzunden
            return
        self.mem_cntr=1000
        self.Q_eval.optimizer.zero_grad()
        max_mem = min(self.mem_cntr, self.mem_size)
        batch = np.random.choice(max_mem, self.batch_size, replace=False) 
        ##chooses random batch_size numbers from 0-patch_size range
        
        batch_index = np.arange(self.batch_size, dtype=np.int32)
        
        state_batch = T.tensor(self.state_memory[batch]).to(self.Q_eval.device)
        new_state_batch = T.tensor(self.new_state_memory[batch]).to(self.Q_eval.device)
        reward_batch = T.tensor(self.reward_memory[batch]).to(self.Q_eval.device)
        terminal_batch = T.tensor(self.terminal_memory[batch]).to(self.Q_eval.device)
        
        action_batch = self.action_memory[batch] ##We dont give actions as input we get all of actions as input
        
        q_eval = self.Q_eval.forward(state_batch)[batch_index, action_batch] 
        q_next = self.Q_eval.forward(new_state_batch)
        q_next[terminal_batch] = 0.0
        
        q_target = reward_batch + self.gamma * T.max(q_next,dim=1)[0]
        
        loss = self.Q_eval.loss(q_target,q_eval).to(self.Q_eval.device)
        loss.backward()
        self.Q_eval.optimizer.step()
        
        self.epsilon = self.epsilon -self.eps_dec if self.epsilon > self.eps_min \
                        else self.eps_min

Copy pasted this cell down there to get learning curve plot

In [14]:
KaggleEnv = ConnectFourGym()
n_actions = KaggleEnv.action_space.n
input_x = KaggleEnv.observation_space.shape[0]*KaggleEnv.observation_space.shape[1]
batch_size_const = 128
n_games_const = 1000

In [15]:
if __name__ == '__main__':
    env = KaggleEnv
    agent = Agent(gamma=0.99, epsilon=1.0, batch_size=batch_size_const, 
                  n_actions=n_actions,eps_end=0.01,input_dims=[input_x],
                  lr=0.0001)
    scores, eps_history = [], []
    n_games = n_games_const
    
    for i in range(n_games):
        score = 0
        done = False
        observation = env.reset()
        while not done:
            action = agent.choose_action(observation)
            observation_,reward,done = env.step(action)
            score +=reward
            agent.store_transition(observation,action,reward,observation_,done)
            agent.learn()
            observation = observation_
        scores.append(score)
        eps_history.append(agent.epsilon)
        
        avg_score = np.mean(scores[-100:])
        
        print('episode ',i, 'score %.2f' %score,
             'average score %.2f' %avg_score,
             'epsilon %.2f' %agent.epsilon)

episode  0 score -1.00 average score -1.00 epsilon 1.00
episode  1 score 1.00 average score 0.00 epsilon 1.00
episode  2 score -1.00 average score -0.33 epsilon 1.00
episode  3 score -10.00 average score -2.75 epsilon 1.00
episode  4 score -10.00 average score -4.20 epsilon 1.00
episode  5 score -1.00 average score -3.67 epsilon 1.00
episode  6 score -1.00 average score -3.29 epsilon 1.00
episode  7 score -1.00 average score -3.00 epsilon 1.00
episode  8 score -1.00 average score -2.78 epsilon 1.00
episode  9 score 1.00 average score -2.40 epsilon 1.00
episode  10 score 1.00 average score -2.09 epsilon 1.00
episode  11 score -1.00 average score -2.00 epsilon 1.00
episode  12 score 1.00 average score -1.77 epsilon 1.00
episode  13 score -1.00 average score -1.71 epsilon 0.99
episode  14 score -1.00 average score -1.67 epsilon 0.99


RuntimeError: expected scalar type Float but found Long